<a href="https://colab.research.google.com/github/ArmaanSeth/Langchain/blob/main/LangGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q langchain langchainhub langgraph langchain-google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.4/246.4 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 13.8 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI

llm=ChatGoogleGenerativeAI(model='gemini-pro', convert_system_message_to_human=True, google_api_key=userdata.get('GOOGLE_API_KEY'), verbose=True)

In [ ]:
from typing import Union, List, Annotated, TypedDict
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
import operator

In [ ]:
class AgentState(TypedDict):
  input: str
  chat_history:list[BaseMessage]
  agent_outcome: Union[AgentAction, AgentFinish, None]
  # return_direct: bool
  intermediate_steps: Annotated[list[tuple[AgentAction,str]], operator.add]

In [ ]:
from langchain.tools import BaseTool, StructuredTool, Tool, tool

In [ ]:
import random

class to_lower_case(BaseTool):
  name="lower_case"
  description="This tool returns the input as all lower case. Input: should be a string of alphabets in uppercase"
  def _run(self, input:str) ->str :
    return input.lower()

class random_number(BaseTool):
    name="random_number"
    description="This tool returns a random number in digits. Input: should be an empty string"
    def _run(self, input:str) -> int:
      return random.randint(0, 100)

tools = [to_lower_case(),random_number()]

In [ ]:
random_number._run(random_number,input='')

42

In [ ]:
to_lower_case._run(to_lower_case,input="23")

'23'

In [ ]:
from langchain_core.prompts import PromptTemplate

template="""
Answer the following questions as best you can.
Question:{input}

You have access to the following tools but you can also choose not to use any tool:

{tools}

To answer the question break the question into smaller task the repeatedly use the below startegy to get the final answer
Use the following format:

Question: what should be done at the moment to make the current answer more suitable for question
Thought: you should always think about what to do. If there is no need to use any tool, simply generate your answer and skip the use of any tool at the moment.
Action: what action to choose from [{tool_names}],it shouold only be the name without ().
Action Input: According to the next action's description, modify the information to be the input to the tool
Observation:The output from action is used to update the current answer.Is the updated answer, the final answer? If not repeat the above steps with same or different tools
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
Thought:{agent_scratchpad}
"""
prompt = PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'],
                        template=template)

# Agent Executor with Langgraph

In [ ]:
from langchain import hub
from langchain.agents import create_react_agent
from langchain_core.agents import AgentActionMessageLog

agent_runnable = create_react_agent(llm, tools, prompt)

In [ ]:
inputs = {"input": "give me a random number and then write in words and make it lower case.",
          "chat_history": [],
          "intermediate_steps":[]}

agent_outcome = agent_runnable.invoke(inputs)
agent_outcome

AgentAction(tool='random_number', tool_input='', log='Action: random_number\nAction Input: ')

In [ ]:
from langgraph.prebuilt.tool_executor import ToolExecutor

tool_executor=ToolExecutor(tools)

In [ ]:
def run_agent(data):
    agent_outcome = agent_runnable.invoke(data)
    return {"agent_outcome": agent_outcome}

def execute_tools(data):
    # Get the most recent agent_outcome - this is the key added in the `agent` above
    agent_action = data["agent_outcome"]
    output = tool_executor.invoke(agent_action)
    print(f"The agent action is {agent_action}")
    print(f"The tool result is: {output}")
    return {"intermediate_steps": [(agent_action, str(output))]}

def should_continue(data):
    # If the agent outcome is an AgentFinish, then we return `exit` string
    # This will be used when setting up the graph to define the flow
    if isinstance(data["agent_outcome"], AgentFinish):
        return "end"
    # Otherwise, an AgentAction is returned
    # Here we return `continue` string
    # This will be used when setting up the graph to define the flow
    else:
        return "continue"

In [ ]:
from langgraph .graph import END, StateGraph

workflow=StateGraph(AgentState)
workflow.add_node("agent", run_agent)
workflow.add_node("action", execute_tools)
workflow.set_entry_point("agent")
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        'continue':'action',
        'end':END
    }
)
workflow.add_edge('action', 'agent')
app=workflow.compile()

In [ ]:
inputs = {"input": "give me a random number and then write in words and make it lower case.", "chat_history":[]}
for s in app.stream(inputs):
    print(list(s.values())[0])
    print("----")

{'agent_outcome': AgentAction(tool='random_number', tool_input='', log='Action: random_number\nAction Input: \n')}
----
The agent action is tool='random_number' tool_input='' log='Action: random_number\nAction Input: \n'
The tool result is: 47
{'intermediate_steps': [(AgentAction(tool='random_number', tool_input='', log='Action: random_number\nAction Input: \n'), '47')]}
----
{'agent_outcome': AgentAction(tool='lower_case', tool_input='47\n', log='Action: lower_case\nAction Input: 47\n')}
----
The agent action is tool='lower_case' tool_input='47\n' log='Action: lower_case\nAction Input: 47\n'
The tool result is: 47

{'intermediate_steps': [(AgentAction(tool='lower_case', tool_input='47\n', log='Action: lower_case\nAction Input: 47\n'), '47\n')]}
----
{'agent_outcome': AgentFinish(return_values={'output': 'forty seven'}, log='Final Answer: forty seven')}
----
{'input': 'give me a random number and then write in words and make it lower case.', 'chat_history': [], 'agent_outcome': AgentFi

In [ ]:
inputs = {"input": "give me a random number and then write in words and make it lower case", "chat_history": []}

res=app.invoke(inputs)

The agent action is tool='random_number' tool_input='' log='Action: random_number\nAction Input: \n'
The tool result is: 49


In [ ]:
res["agent_outcome"].return_values['output']

'forty-nine'

In [ ]:
inputs = {"input": "Explain is AI?", "chat_history": []}

res=app.invoke(inputs)

In [ ]:
res["agent_outcome"].return_values['output']

'AI stands for Artificial Intelligence, which is a branch of computer science that seeks to understand and create intelligent entities. AI research has the potential to revolutionize many aspects of human life, from healthcare to transportation to finance.'

# Chat Agent Executor

In [ ]:
from langgraph.prebuilt import ToolExecutor

tool_executor=ToolExecutor(tools)

In [ ]:
from langchain.tools.render import format_tool_to_openai_function

functions=[format_tool_to_openai_function(t) for t in tools]
model=llm.